# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [3]:
# your code here
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme=84&offset=84&sort=salet&limi'
url


'https://www.zalando.fr/api/catalog/articles?categories=promo-homme=84&offset=84&sort=salet&limi'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [2]:
# your code here
import json
import pandas as pd
from urllib.request import urlopen
from pandas.io.json import json_normalize
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme=84&offset=84&sort=salet&limi'


In [12]:
response = urlopen(url)
results = json.load(response)
print(results)

{'total_count': 0, 'pagination': {'page_count': 1, 'current_page': 2, 'per_page': 84}, 'sort': 'popularity', 'articles': [], 'query_path': '/promo-homme=84/?p=2', 'previous_page_path': '/promo-homme=84/', 'query_params': {'categories': 'promo-homme=84', 'sort': 'popularity', 'offset': '84', 'limit': '84'}, 'page_gender': 'unisex', 'premium': False, 'appliedSort': None, 'filters': [], 'total_article_count': 0, 'assortment_entry_points': [], 'plusStatus': 'unknown', 'categoryTree': [], 'sortingKeys': [{'type': 'popularity', 'dir': 'desc', 'label': 'Préférés'}, {'type': 'activation_date', 'dir': 'desc', 'label': 'Nouveautés'}, {'type': 'price_asc', 'dir': 'asc', 'label': 'Prix croissant'}, {'type': 'price_desc', 'dir': 'desc', 'label': 'Prix décroissant'}, {'type': 'sale', 'dir': 'desc', 'label': 'Promos'}], 'breadcrumbs': [], 'querySemantics': {'positive_filters': [{'key': 'categories', 'url_key': 'category', 'kind': 'values', 'values': [{'key': 'promo-homme=84', 'url_key': 'promo-homme=

In [8]:
flattened_data = json_normalize(results)
print(flattened_data)


   total_count        sort articles            query_path previous_page_path  \
0            0  popularity       []  /promo-homme=84/?p=2   /promo-homme=84/   

  page_gender  premium appliedSort filters  total_article_count  ...  \
0      unisex    False        None      []                    0  ...   

  iconPaths.filters.standard_delivery_filter  \
0                            icons/truck.svg   

  iconPaths.filters.fast_delivery_filter iconPaths.filters.zalando_plus  \
0                   icons/truck-fast.svg         icons/plus-short-1.svg   

  iconPaths.mobileFilters.standard_delivery_filter  \
0                                  icons/truck.svg   

  iconPaths.mobileFilters.fast_delivery_filter  \
0                         icons/truck-fast.svg   

   iconPaths.mobileFilters.zalando_plus iconPaths.flags.slow_delivery_flag  \
0                icons/plus-short-1.svg                    icons/clock.svg   

  iconPaths.flags.fast_delivery_flag  iconPaths.flags.plus_delivery_flag  \
0  

In [14]:
# total number of pages
total_pages=results['pagination']['page_count']
total_pages


1

In [15]:
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}&sort=sale'
    response = urlopen(url)
    results = json.load(response)
    flattened_data = pd.json_normalize(results)
    flattened_data1 = pd.json_normalize(flattened_data.articles[0])
    flattened_data1 = flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,outfits
sku,,,,,,,,,,,,,,,,,
TO122O08I-Q11,LOGO TEE - T-shirt imprimé - black,"[XS, S, M, L, XL, XXL, 3XL]",tommy-hilfiger-logo-tee-t-shirt-imprime-to122o...,[{'path': 'spp-media-p1/6a1582f6fde836a5917156...,Tommy Hilfiger,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"39,95 €","27,95 €",False,False,False,False,NaN
HY554B01L-Q11,Masque en tissu - black,[One Size],hype-masque-en-tissu-black-hy554b01l-q11,[{'path': 'spp-media-p1/c15457b8f18532a086b55a...,Hype,False,[],"[{'key': 'discountRate', 'value': '-70%', 'tra...",accessoires,[],"15,00 €","4,50 €",False,False,False,False,NaN
AD122O0GM-Q11,TECH TEE - T-shirt imprimé - black,"[XS, S, M, L, XL, XXL]",adidas-originals-tech-tee-t-shirt-imprime-ad12...,[{'path': 'spp-media-p1/48b9327c2f6433cc82cee6...,adidas Originals,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],"29,95 €","22,45 €",True,False,True,False,NaN
JA222E0QZ-K11,JJIMARCO JJPHIL - Pantalon classique - navy b...,"[28x30, 29x30, 29x32, 30x32, 32x30, 32x32, 32x...",jack-and-jones-jjimarco-jjphil-pantalon-classi...,[{'path': 'spp-media-p1/5dfb794c7c8b46fe8adbf8...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -50...",clothing,[],"39,99 €","20,00 €",True,False,True,False,NaN
JA222E0F5-K11,MARCO BOWIE - Chino - navy,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 30x...",jack-and-jones-marco-bowie-chino-navy-ja222e0f...,[{'path': 'spp-media-p1/33fc78cacde137fc8cecd1...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],"39,99 €","29,99 €",True,False,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PI922S038-A11,Sweat à capuche - white,"[XS, S, M, L, XL, XXL]",pier-one-sweatshirt-white-pi922s038-a11,[{'path': 'spp-media-p1/db071f03a64d3629a1b95e...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"24,99 €","20,09 €",False,False,False,False,NaN
JA222T0NJ-G11,Veste mi-saison - tango red,"[S, M, L]",jack-and-jones-veste-dhiver-tango-red-ja222t0n...,[{'path': 'spp-media-p1/3e1a50002e8e395395e3e4...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",clothing,[],"49,99 €","25,00 €",False,False,False,False,NaN
JA222O32B-G12,JJELOGO BLOCKING TEE - T-shirt imprimé - tango...,"[XS, S, M, L, XL, XXL]",jack-and-jones-jjelogo-blocking-tee-t-shirt-im...,[{'path': 'spp-media-p1/cf5ccef4201739f993da5d...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"14,99 €","10,49 €",False,False,False,False,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [16]:
# your code here
df.brand_name.value_counts().index[0]

'Jack & Jones'